<h1 style="text-align:center;font-size:40px;">Tools 1 Final Project</h1>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<p style="text-align:center;">Anthony Molieri<br>
Leo Walker</p>
<br>
<br>
<br>
<br>

# Dataset and Motivation

The data used for these analyses were scraped from Indeed and Glassdoor. The data from Indeed contains job descriptions for open Data Scientist positions. The Glassdoor data includes employee generated ratings on benefits and perks for many of the companies present in the Indeed dataset. The datasets were then combined in order to give an idea of how employees rank the benefits at these companies.

*The Indeed data was scraped over ... pulling job descriptions for Data Scientists in 25 cities throughout the US. **Leo** Add Correct Details, here about the Indded data and scraper*

The Glassdoor scraper was designed to pull company names from the the job description dataset. It then looped over the list of employers, searched, and scraped ratings from the benefits page for companies that were found. This data was then added to it's own table in the same database. The scraper was designed to be run multiple times, only searching for benefits ratings that were not already present in the database. 

# Task Definition and Research Question
 
The purpose of this project was to help give a better idea of how employees at companies with open positions rate their benefits. This can help give insight into how desirable a company is as an employer.

# Literature Review

*Add -- I'm not really sure what to add here. Maybe something about turnover in data science? Companies not having great benefits?*

# Data Cleaning

In [2]:
# imports
import numpy as np
import pandas as pd
import sqlite3
import re

In [3]:
# Initialize both tables
cnct = sqlite3.connect('Data/jobs')

indeed_df = pd.read_sql_query('SELECT * FROM indeed_jobs;', cnct)

glassdoor_df = pd.read_sql_query('SELECT companies, type, rating, count_of_ratings FROM ratings;', cnct)

## Cleaning Indeed Data

In [4]:
# # Pull missing salary data from description
# salaryFix = indeed_df[['job_desc', 'est_salary']].copy()

# salaryFix['check1'] = salaryFix['job_desc'].str.contains("\$")
# salaryFix['check2'] = salaryFix['job_desc'].str.contains("salary")

# I'm not great at regex but let's try this
# Only look for salary data when it's missint in 'est_salary' 
indeed_df['salary_extract'] = np.where(indeed_df['est_salary'] == "No Estimated Salary",indeed_df['job_desc'].str.extract("((\$)(([0-9])([0-9])([0-9])(,)([0-9])([0-9])([0-9])|([0-9])([0-9])(,)([0-9])([0-9])([0-9]))( to |-| - )(\$|)(([0-9])([0-9])([0-9])(,)([0-9])([0-9])([0-9])|([0-9])([0-9])(,)([0-9])([0-9])([0-9]))|(\$)([0-9][0-9][0-9],[0-9][0-9][0-9])|(\$)([0-9][0-9],[0-9][0-9][0-9]))")[0], np.NaN)

In [5]:
def set_job_category(job_title):
    regex = re.compile('[^a-zA-Z]')
    clean_title = regex.sub('', job_title)
    clean_title = clean_title.lower()
    if all(x in clean_title for x in ["data", "scien"]):
        return "Data Scientist"
    elif all(x in clean_title for x in ["business", "analy"]):
        return "Business Intelligence Analyst"
    elif all(x in clean_title for x in ["bi", "analy"]):
        return "Business Intelligence Analyst"
    elif all(x in clean_title for x in ["data", "analy"]):
        return "Data Analyst"
    elif all(x in clean_title for x in ["machine", "learning"]):
        return "Machine Learning Engineer"
    elif all(x in clean_title for x in ["data", "engineer"]):
        return "Data Engineer"
    elif all(x in clean_title for x in ["software", "engin"]):
        return "Software Engineer"
    elif all(x in clean_title for x in ["architect"]):
        return "Data Architect"
    else:
        return "Other"
indeed_df['job_category'] = indeed_df.loc[:, 'job_title'].apply(set_job_category)

In [6]:
indeed_df.loc[:,'state'] = indeed_df.loc[:, 'company_location'].str.extract(r', ([A-Z]{2})')

In [7]:
indeed_df[indeed_df['est_salary']!='No Estimated Salary'].sample(5)

,index,job_title,company_name,company_location,est salary,job_href,job_desc,est_salary,retrieve_date,salary_extract,job_category,state
13499,2,Software Engineer,Magnite,"Broomfield, CO",None,https://www.indeed.com/company/Magnite-Austral...,Join Magnite as a Software Engineer - Full St...,"$80,000 - $110,000 a year",2022-04-04,NaN,Software Engineer,CO
6069,11,Senior Data Engineer / Senior Machine Learning...,Vail Resorts,"Broomfield, CO",None,https://www.indeed.com/company/Enterprise-Solo...,Position : Senior Data Engineer / Senior Machi...,"$90,000 - $120,000 a year",2022-02-26,NaN,Machine Learning Engineer,CO
2185,0,Staff Data Engineer,Slack,"San Francisco, CA",None,https://www.indeed.com/rc/clk?jk=9a1ece6e9b69c...,"To get the best candidate experience, please c...","$166,000 a year",2022-02-12,NaN,Data Engineer,CA
5870,7,Business Analyst - Remote - Strategic Product ...,Wilbur-Ellis Company LLC,"Remote in Aurora, CO",None,https://www.indeed.com/rc/clk?jk=36c3fb15b8950...,Wilbur-Ellis is one of the largest family-owne...,"$85,000 - $110,000 a year",2022-02-26,NaN,Business Intelligence Analyst,CO
9069,2,Senior Software Engineer (Supply Chain Technol...,Nordstrom Inc,"Remote in Denver, CO",None,https://www.indeed.com/rc/clk?jk=149fa4de46f32...,Job Description The Senior Software Engineer w...,"$114,000 - $176,500 a year",2022-03-14,NaN,Software Engineer,CO


In [8]:
def salary_low(row):
    if row =='No Estimated Salary':
        return None
    salary = str(row).split('-')
    # print(f'Salary={salary}')
    pattern = r'([0-9]*)'
    if len(salary)>1:
        low = int(''.join(re.findall(pattern, salary[0])))
        high = int(''.join(re.findall(pattern, salary[1])))
    else:
        low = high = int(''.join(re.findall(pattern, salary[0])))
    # print(f'low={low}, high={high} ')
    return low

def salary_high(row):
    if row =='No Estimated Salary':
        return None
    salary = str(row).split('-')
    # print(f'Salary={salary}')
    pattern = r'([0-9]*)'
    if len(salary)>1:
        low = int(''.join(re.findall(pattern, salary[0])))
        high = int(''.join(re.findall(pattern, salary[1])))
    else:
        low = high = int(''.join(re.findall(pattern, salary[0])))
    # print(f'low={low}, high={high} ')
    return high

indeed_df.loc[:,['salary_low']] = indeed_df.loc[:,'est_salary'].apply(salary_low)
indeed_df.loc[:,['salary_high']] = indeed_df.loc[:,'est_salary'].apply(salary_high)

In [13]:
def pay_rate(row):
    regex = re.compile('[.[0-9]]')
    clean_salary = regex.sub('', row)
    clean_salary = clean_salary.lower()
    print(clean_salary)
    if any(x in clean_salary for x in ["year"]):
        return "annual"
    elif any(x in clean_salary for x in ["month"]):
        return "month"
    elif any(x in clean_salary for x in ["day"]):
        return "day"
    elif any(x in clean_salary for x in ["hour"]):
        return "hour"
    else:
        return "Other"

indeed_df['pay_rate'] = indeed_df.loc[:, 'est_salary'].apply(pay_rate)

no estimated salary
no estimated salary
no estimated salary
no estimated salary
no estimated salary
no estimated salary
$56,000 - $106,080 a year
no estimated salary
no estimated salary
no estimated salary
$72,500 - $137,400 a year
no estimated salary
no estimated salary
no estimated salary
no estimated salary
no estimated salary
no estimated salary
no estimated salary
no estimated salary
no estimated salary
no estimated salary
no estimated salary
no estimated salary
no estimated salary
no estimated salary
no estimated salary
$100,800 - $230,200 a year
no estimated salary
no estimated salary
no estimated salary
no estimated salary
no estimated salary
$110,500 a year
no estimated salary
no estimated salary
no estimated salary
no estimated salary
no estimated salary
no estimated salary
no estimated salary
no estimated salary
no estimated salary
no estimated salary
no estimated salary
no estimated salary
no estimated salary
no estimated salary
no estimated salary
no estimated salary
no es

In [17]:
indeed_df[indeed_df['est_salary']!='No Estimated Salary'].sample(10)

,index,job_title,company_name,company_location,est salary,job_href,job_desc,est_salary,retrieve_date,salary_extract,job_category,state,salary_low,salary_high,pay_rate
15771,2,Associate Data Scientist,MissionWired,"Washington, DC+1 location",None,https://www.indeed.com/company/Anne-Lewis-Stra...,"WASHINGTON, DC - REMOTE FLEXIBILITY /DATA SCI...","$55,000 - $75,000 a year",2022-04-10,NaN,Data Scientist,DC,55000.0,75000.0,annual
2867,11,Behavioral Health Business Analyst,State of Colorado Job Opportunities,"Remote in Denver, CO+2 locations",None,https://www.indeed.com/rc/clk?jk=7d63853bb11be...,Department Information STATE OF COLORADO RESI...,"$5,428 - $6,687 a month",2022-02-12,NaN,Business Intelligence Analyst,CO,5428.0,6687.0,month
12876,6,Machine Learning Lead with Python Experience,X2logicsstaffingsolution.com,"Austin, TX",None,https://www.indeed.com/company/x2logicsstaffin...,Job Description: Machine Learning Experienc...,"$100,000 - $130,000 a year",2022-04-03,NaN,Machine Learning Engineer,TX,100000.0,130000.0,annual
8484,2,Business Analyst - 4212656,Accenture,"Raleigh, NC 27609 (Falls of Neuse area)",None,https://www.indeed.com/rc/clk?jk=f312277ee6f4a...,Accenture Flex offers you the flexibility of l...,$68 - $78 an hour,2022-03-05,NaN,Business Intelligence Analyst,NC,68.0,78.0,hour
8767,0,Financial Data Analyst,Black Knight,"Lakewood, CO",None,https://www.indeed.com/rc/clk?jk=834db3f15593d...,Black Knight is the premier provider of integr...,"$65,000 - $80,000 a year",2022-03-14,NaN,Data Analyst,CO,65000.0,80000.0,annual
17002,5,BUSINESS ANALYSIS SUPERVISOR,State of Arizona,"Phoenix, AZ 85007 (Central City area)",None,https://www.indeed.com/rc/clk?jk=b77b18442c699...,The Division of Health Care Management (DH...,"$68,000 - $70,871 a year",2022-04-11,NaN,Business Intelligence Analyst,AZ,68000.0,70871.0,annual
2501,1,Business Analyst,US Federal Acquisition Service,"Seattle, WA",None,https://www.indeed.com/rc/clk?jk=69b99e3509c85...,"Duties We are currently filling one vacancy, b...","$95,973 - $124,764 a year",2022-02-12,NaN,Business Intelligence Analyst,WA,95973.0,124764.0,annual
6013,2,Statistician (Health),US Substance Abuse and Mental Health Services...,"Rockville, MD",None,https://www.indeed.com/rc/clk?jk=7c6ff6280fa3f...,Help This job is open to Career transition (CT...,"$89,834 - $138,868 a year",2022-02-26,NaN,Other,MD,89834.0,138868.0,annual
16038,14,Data Analyst,Piper Companies,"Boston, MA",None,https://www.indeed.com/rc/clk?jk=9492d4cc2e234...,"Piper Companies is seeking a Data Analyst, t...","$75,000 - $105,000 a year",2022-04-11,NaN,Data Analyst,MA,75000.0,105000.0,annual
7008,3,Health Disparities Data Analyst,Dept of State Health Services,"Austin, TX 78756 (Brentwood area)",None,https://www.indeed.com/rc/clk?jk=e105a43d45335...,Through a CDC-funded initiative striving to re...,"$5,785 - $7,784 a month",2022-03-05,NaN,Data Analyst,TX,5785.0,7784.0,month


In [22]:
import re

In [24]:
pattern = r'([0-9]*)'
s = '$75,000 a year'
''.join(re.findall(pattern, s))

'75000'

In [25]:
str('$75,000 a year').split('-')[0]

'$75,000 a year'

## Cleaning Glassdoor Data

# Visualizations